In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
from datasets import OrganoidDataset
data = OrganoidDataset()

/home/egor/PycharmProjects/deep_dr/venv/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [3]:
print(data.train[0].shape,data.train[1].shape)
print(data.val[0].shape,data.val[1].shape)
print(data.test[0].shape,data.test[1].shape)

torch.Size([820733, 41]) torch.Size([820733, 2])
torch.Size([234495, 41]) torch.Size([234495, 2])
torch.Size([117248, 41]) torch.Size([117248, 2])


In [4]:
from models import VanillaVAE
model = VanillaVAE(in_features=41, hidden_dims = [41,41,41], latent_dim= 2).to('cuda')

In [5]:
X_train,y_train = data.train
X_val,y_val = data.val

In [6]:
from torch import optim

optimizer = optim.Adam(model.parameters(),
                       lr=0.005,
                       weight_decay=0.0,
                       )

In [7]:
from tqdm import tqdm

pbar = tqdm(range(1000))

for epoch in pbar:
    optimizer.zero_grad()
    outputs = model.forward(X_train)
    loss = model.loss_function(*outputs, M_N=0.0025)['loss']
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        with torch.no_grad():
            outputs = model.forward(X_val)
            loss = model.loss_function(*outputs, M_N=0.0025)

            pbar.set_description(f"Epoch: {epoch}, "
                             f"Loss: {loss['loss']}, "
                             f"Reconstruction loss: {loss['Reconstruction_Loss']}, "
                             f"KL loss: {loss['KLD']}", refresh=True)

print('Finished Training')

Epoch: 140, Loss: 0.49964088201522827, Reconstruction loss: 0.4841030538082123, KL loss: -6.215127468109131:  15%|█▍        | 146/1000 [00:16<01:36,  8.88it/s] 


KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), 'logs/VanillaVAE/latent=2_epoch=10000_layers=41,41,41.pth')

In [6]:
from sklearn.decomposition import PCA
import numpy as np
pca = PCA(n_components=10)
train_data = X_train.cpu()
pca.fit(train_data)
print(pca.explained_variance_ratio_)
print(np.cumsum(pca.explained_variance_ratio_))

[0.35452994 0.18482029 0.05850656 0.04639866 0.03060609 0.02742573
 0.02369693 0.02089108 0.01877983 0.01733262]
[0.35452994 0.53935023 0.59785679 0.64425544 0.67486153 0.70228726
 0.72598418 0.74687526 0.76565509 0.78298772]


In [7]:
from torch.nn import functional as F
val_data = X_val.cpu()
for i in range(10):
    reduced_dim = np.pad(pca.transform(val_data)[:,:i],((0,0),(0,10-i)))
    loss = F.mse_loss(val_data, torch.from_numpy(pca.inverse_transform(reduced_dim)))
    print(f"Components: {i}, MSE:{loss}")

Components: 0, MSE:1.3534091070398027
Components: 1, MSE:0.8742205681688913
Components: 2, MSE:0.6241996179389656
Components: 3, MSE:0.544869697229827
Components: 4, MSE:0.48213737385965566
Components: 5, MSE:0.44071113664262956
Components: 6, MSE:0.4035927459006117
Components: 7, MSE:0.3716002847620463
Components: 8, MSE:0.3433178042269015
Components: 9, MSE:0.31782799266804174


In [ ]:
with torch.no_grad():
    latent_val = model.reparameterize(*model.encode(X_val)).to('cpu')
    outputs = model.forward(X_val)
    loss = model.loss_function(*outputs, M_N=0.0025)
print(loss)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
%matplotlib inline

In [ ]:
from datasets import CellType
reduced_dims = pd.DataFrame(y_val.cpu()).rename({0:"Cell type",1:"Day"},axis=1)
reduced_dims['Cell type'].replace({i.value: i.name for i in CellType}, inplace=True)

reduced_dims["PC1"]= pca.transform(X_val.cpu())[:,0]
reduced_dims["PC2"]= pca.transform(X_val.cpu())[:,1]
reduced_dims["VAE1"] = latent_val[:,0]
reduced_dims["VAE2"] = latent_val[:,1]
reduced_dims

In [ ]:
sns.lmplot( x="PC1", y="PC2",
  data=reduced_dims.head(10000),
  fit_reg=False,
  hue='Cell type', # color by cluster
  legend=True,
  scatter_kws={"s": 5})

In [ ]:
sns.lmplot( x="VAE1", y="VAE2",
  data=reduced_dims.head(10000),
  fit_reg=False,
  hue='Cell type', # color by cluster
  legend=True,
  scatter_kws={"s": 5})

In [ ]:
sns.lmplot( x="PC1", y="PC2",
  data=reduced_dims.head(10000),
  fit_reg=False,
  hue='Day', # color by cluster
  legend=True,
  scatter_kws={"s": 5})

In [ ]:
sns.lmplot( x="VAE1", y="VAE2",
  data=reduced_dims.head(10000),
  fit_reg=False,
  hue='Day', # color by cluster
  legend=True,
  scatter_kws={"s": 5})

In [ ]:
from datasets import OrganoidDataset

in_features=41
latent_dim=2
hidden_dims = [41,32,16]
kld_weight=0.0025
output_dir = './logs/VanillaVAE/'
lr = 0.005
weight_decay = 0.0
epochs=1000
save_loss_every_n_epochs=10
dataset=OrganoidDataset

In [ ]:
import torch
from models import VanillaVAE
print(f"CUDA available:{torch.cuda.is_available()}")
data = dataset()
model = VanillaVAE(in_features=in_features, hidden_dims=hidden_dims, latent_dim=latent_dim).to('cuda')

In [ ]:
from torch import optim
from tqdm import tqdm
for _ in range(1):
    X_train, y_train = data.train
    X_val, y_val = data.val

    optimizer = optim.Adam(model.parameters(),
                           lr=lr,
                           weight_decay=weight_decay,
                           )

    loss_list = list()
    for epoch in tqdm(range(1, epochs+1)):
        optimizer.zero_grad()
        outputs = model.forward(X_train)
        loss = model.loss_function(*outputs, M_N=kld_weight)
        loss['loss'].backward()
        optimizer.step()
        if epoch % save_loss_every_n_epochs == 0 or epoch == epochs:
            with torch.no_grad():
                loss_dict = dict()
                loss_dict['epoch'] = epoch
                loss_dict['train_loss'] = loss['loss'].to('cpu').numpy().item()
                loss_dict['train_rec_loss'] = loss['Reconstruction_Loss'].to('cpu').numpy().item()
                loss_dict['train_kld'] = loss['KLD'].to('cpu').numpy().item()

                outputs_val = model.forward(X_val)
                loss_val = model.loss_function(*outputs_val, M_N=kld_weight)

                loss_dict['val_loss'] = loss_val['loss'].to('cpu').numpy().item()
                loss_dict['val_rec_loss'] = loss_val['Reconstruction_Loss'].to('cpu').numpy().item()
                loss_dict['val_kld'] = loss_val['KLD'].to('cpu').numpy().item()

                loss_list.append(loss_dict)

    print('Finished Training')

In [ ]:
import pandas as pd
pd.DataFrame(loss_list)

In [ ]:
with torch.no_grad():
    latent_val = model.reparameterize(*model.encode(X_val)).to('cpu')
    reconstructed_val = model.generate(X_val).to('cpu')

In [ ]:
from experiment import run_vae

In [ ]:
run_vae(in_features=41,epochs=50000,hidden_dims=[41,32,16,8,4],save_loss_every_n_epochs=100)

In [1]:
import importlib
importlib.import_module('.c', 'a.b')

ModuleNotFoundError: No module named 'a'

In [3]:
str(tuple([1,2,3,4,5]))

'(1, 2, 3, 4, 5)'

['"(128, 128)"',
 '"(128, 128, 128)"',
 '"(128, 128, 128, 128)"',
 '"(128, 128, 128, 128, 128)"',
 '"(128, 128, 128, 128, 128, 128)"',
 '"(128, 128, 128, 128, 128, 128, 128)"',
 '"(128, 128, 128, 128, 128, 128, 128, 128)"']

In [1]:
from models import BetaVAE

In [2]:
import pickle
with open('/data/PycharmProjects/cytof_benchmark/logs/BetaVAE/exp_5/run_5/config.pkl', 'rb') as file:
    config_best = pickle.load(file)
model_best = BetaVAE(config_best).to(config_best.device)

In [4]:
def test(model,val_dataloader):
    with torch.no_grad():
        model.eval()
        mse,kld,loss = list(),list(),list()
        for X_batch in val_dataloader:
            loss_dict = dict()
            outputs = model.forward(X_batch)
            losses = model.loss_function(*outputs)
            for key in losses.keys():
                loss_dict[key] = losses[key].item() * X_batch.shape[0]
            mse.append(loss_dict['MSE'])
            kld.append(loss_dict['KLD'])
            loss.append(loss_dict['loss'])
    data_len = sum(len(batch) for batch in val_dataloader)
    return sum(mse)/data_len,sum(kld)/data_len, sum(loss)/data_len

In [6]:
import torch
from datasets import OrganoidDataset

data = OrganoidDataset()

X_train, y_train = data.train
X_val, y_val = data.val
X_train_batches = torch.split(X_train, split_size_or_sections=1024)
X_val_batches = torch.split(X_val, split_size_or_sections=1024)

In [7]:
checkpoint = torch.load('/data/PycharmProjects/cytof_benchmark/logs/BetaVAE/exp_5/run_5/model.pth')

In [9]:
model_best.load_state_dict(checkpoint)

<All keys matched successfully>

In [10]:
test(model_best,X_val_batches)

(0.2957925220464002, -10.51027182543638, 0.32206820037098627)